# Train Low Test Models

This notebook is a streamlined notebook for generating minima of low test accuracy through three different means:
- Dataset Poisoning
- Adding Noise to Data
- Decreasing Dataset Sizes

## Imports

In [1]:
# Standard library
import copy
import os
import sys
import time

# Third-party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Local package imports
from minima_volume.dataset_funcs import (
    prepare_datasets,
    save_dataset,
    save_model,
)
from minima_volume.train_funcs import evaluate, train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:

# ==============================
# Base Input Parameters
# ==============================
# --- SEEDS ---
data_seed = 10            
model_seed = 0           

# --- Training configuration ---
epochs = 5000            

# --- Dataset configuration ---
base_data_size = (int)(0.02*9409)
dataset_type = "data"   
dataset_quantities = [0, (int)(0.03*9409), (int)(0.08*9409), 
                      (int)(0.18*9409), (int)(0.28*9409),
                      (int)(0.33*9409),
                      (int)(0.38*9409), (int)(0.48*9409),
                     (int)(0.58*9409), (int)(0.68*9409),] # modulo arithmetic dataset sizes
# --- Output configuration ---
base_output_dir = ""     
save_generated_dataset = True   
save_generated_models = True    



In [3]:
print (base_data_size)
print (dataset_quantities)

188
[0, 282, 752, 1693, 2634, 3104, 3575, 4516, 5457, 6398]


## Model + Dataset Specific Code

This is for specific code.

In [4]:
# User specifies the model module name
from minima_volume.models import modulo_arithmetic_model_data as model_module
modulus = 97
# Generate dataset
x_base, y_base, x_test, y_test = model_module.get_dataset(
    modulus = modulus,
    device = device,
)

# MNIST specific initialization parameters
hidden_dims = [250]

# Grab model
model_template = model_module.get_model(N = modulus, hidden_dims=hidden_dims, device=device, seed=model_seed)

# Grab loss and metrics
loss_fn = model_module.get_loss_fn()
other_metrics = model_module.get_additional_metrics()

## Training

We generate the various datasets used to train our models here, before training them. We record the losses, and what each model was trained on.

In [5]:
# ==============================
# Prepare datasets
# ==============================
x_base_train, y_base_train, x_additional, y_additional = prepare_datasets(
    x_base=x_base,
    y_base=y_base,
    dataset_type=dataset_type,
    dataset_quantities=dataset_quantities,
    base_data_size=base_data_size,
    data_seed=data_seed,
    seed_1=None,
    seed_2=None,
)

x_base_train = x_base_train.to(device)
y_base_train = y_base_train.to(device)
x_additional = x_additional.to(device)
y_additional = y_additional.to(device)
x_test = x_test.to(device)
y_test = y_test.to(device)

# ==============================
# Training loop
# ==============================
all_models = []

for additional_data in dataset_quantities:
    # Assemble training dataset
    x_train = torch.cat([x_base_train, x_additional[:additional_data]], dim=0)
    y_train = torch.cat([y_base_train, y_additional[:additional_data]], dim=0)

    # Initialize model (defined in the model-specific file)
    torch.manual_seed(model_seed)
    model = copy.deepcopy(model_template)
    optimizer = optim.AdamW(model.parameters(), lr=1e-3)
    batch_size = len(x_train)

    # Train model
    train_loss, train_other_metrics, test_loss, test_other_metrics = train(
        model = model,
        x_train = x_train, y_train = y_train,
        x_test = x_test, y_test = y_test,
        loss_fn = loss_fn,
        metrics = other_metrics,
        optimizer = optimizer,
        epochs=epochs,
        batch_size=batch_size,
        verbose_every=100,
    )
    
    # Build dictionary dynamically for additional metrics
    train_metrics_dict = {}
    test_metrics_dict = {}
    if train_other_metrics is not None:
        # train_other_metrics is a list of dicts per epoch
        for metric_name in train_other_metrics[0].keys():  # keys from first epoch
            train_metrics_dict[f"train_{metric_name}"] = [m[metric_name] for m in train_other_metrics]
            test_metrics_dict[f"test_{metric_name}"] = [m[metric_name] for m in test_other_metrics]
    
    # Store results
    trained_model = {
        "model": model,
        "train_loss": train_loss,
        "test_loss": test_loss,
        "additional_data": additional_data,
        "dataset_type": dataset_type,
        **train_metrics_dict,  # dynamically include additional metrics
        **test_metrics_dict,
    }
    
    all_models.append(trained_model)

    print(f"Completed training with {additional_data} additional samples of {dataset_type}")

    # Free memory (important for large GPU datasets)
    del x_train, y_train
    torch.cuda.empty_cache()


Epoch 1/5000: Train Loss 0.0125 | Test Loss 0.0120 | accs Train 0.0000 Test 0.0099


Epoch 100/5000: Train Loss 0.0014 | Test Loss 0.0156 | accs Train 1.0000 Test 0.0204


Epoch 200/5000: Train Loss 0.0000 | Test Loss 0.0180 | accs Train 1.0000 Test 0.0203


Epoch 300/5000: Train Loss 0.0000 | Test Loss 0.0183 | accs Train 1.0000 Test 0.0203


Epoch 400/5000: Train Loss 0.0000 | Test Loss 0.0184 | accs Train 1.0000 Test 0.0203


...

Epoch 4600/5000: Train Loss 0.0007 | Test Loss 0.0007 | accs Train 1.0000 Test 1.0000


Epoch 4700/5000: Train Loss 0.0007 | Test Loss 0.0007 | accs Train 1.0000 Test 1.0000


Epoch 4800/5000: Train Loss 0.0007 | Test Loss 0.0007 | accs Train 1.0000 Test 1.0000


Epoch 4900/5000: Train Loss 0.0007 | Test Loss 0.0007 | accs Train 1.0000 Test 1.0000


Epoch 5000/5000: Train Loss 0.0007 | Test Loss 0.0007 | accs Train 1.0000 Test 1.0000
Completed training with 6398 additional samples of data


## Training Summary

In [6]:
# ====================================
# Summary of Training Results
# ====================================
print("=== True Generalization ===")
for model_data in all_models:
    model = model_data["model"]
    additional_data = model_data["additional_data"]

    test_loss, test_metrics = evaluate(
        model=model,
        x_test=x_test,
        y_test=y_test,
        loss_fn=loss_fn,
        metrics=other_metrics
    )

    metrics_str = " | ".join([f"{name}: {val:.4f}" for name, val in test_metrics.items()])
    print(
        f"{additional_data:>4} samples | "
        f"Test Loss: {test_loss:.4f}" + (f" | {metrics_str}" if metrics_str else "")
    )

print("\n=== Model Diagnostics by Training Data ===")
for additional_data in dataset_quantities:
    # Build dataset with this many additional samples
    x_train = torch.cat([x_base_train, x_additional[:additional_data]], dim=0)
    y_train = torch.cat([y_base_train, y_additional[:additional_data]], dim=0)

    print(f"\nDataset type: {dataset_type}, additional samples: {additional_data}")

    for model_data in all_models:
        model = model_data["model"]
        model_additional_data = model_data["additional_data"]

        train_loss, train_metrics = evaluate(
            model=model,
            x_test=x_train,
            y_test=y_train,
            loss_fn=loss_fn,
            metrics=other_metrics
        )

        metrics_str = " | ".join([f"{name}: {val:.4f}" for name, val in train_metrics.items()])
        print(
            f" Model {model_additional_data:>4} | "
            f"Train Loss: {train_loss:.4f}" + (f" | {metrics_str}" if metrics_str else "")
        )

    # Free memory if large
    del x_train, y_train
    torch.cuda.empty_cache()

=== True Generalization ===
   0 samples | Test Loss: 0.0181 | accs: 0.0203
 282 samples | Test Loss: 0.0264 | accs: 0.0514
 752 samples | Test Loss: 0.0381 | accs: 0.1051
1693 samples | Test Loss: 0.0186 | accs: 0.2041
2634 samples | Test Loss: 0.0081 | accs: 0.9736
3104 samples | Test Loss: 0.0019 | accs: 1.0000
3575 samples | Test Loss: 0.0012 | accs: 1.0000
4516 samples | Test Loss: 0.0007 | accs: 1.0000
5457 samples | Test Loss: 0.0006 | accs: 1.0000
6398 samples | Test Loss: 0.0007 | accs: 1.0000

=== Model Diagnostics by Training Data ===

Dataset type: data, additional samples: 0
 Model    0 | Train Loss: 0.0000 | accs: 1.0000
 Model  282 | Train Loss: 0.0000 | accs: 1.0000
 Model  752 | Train Loss: 0.0006 | accs: 1.0000
 Model 1693 | Train Loss: 0.0038 | accs: 1.0000
 Model 2634 | Train Loss: 0.0036 | accs: 1.0000
 Model 3104 | Train Loss: 0.0010 | accs: 1.0000
 Model 3575 | Train Loss: 0.0008 | accs: 1.0000
 Model 4516 | Train Loss: 0.0006 | accs: 1.0000
 Model 5457 | Train L

### Model + Data Specific Verification

In [7]:
model_module.verify_model_results(
    all_models=all_models,
    x_base_train=x_base_train,
    y_base_train=y_base_train,
    x_additional=x_additional,
    y_additional=y_additional,
    x_test=x_test,
    y_test=y_test,
    dataset_quantities=dataset_quantities,
    dataset_type=dataset_type,
)

## Model Saving

In [8]:
# ====================================
# Save Datasets and Models
# ====================================
output_folder = "models_and_data"
# Save dataset (Possible to skip)
if save_generated_dataset:
    save_dataset(
        folder=output_folder,
        filename="dataset.pt",
        x_base_train=x_base_train,
        y_base_train=y_base_train,
        x_additional=x_additional,
        y_additional=y_additional,
        x_test=x_test,
        y_test=y_test,
        dataset_quantities=dataset_quantities,
        dataset_type=dataset_type,
    )
    print(f"Saved dataset to {output_folder}/dataset.pt")

# Save trained models
if save_generated_models:
    for model_data in all_models:
        filename = f"model_additional_{model_data['additional_data']}.pt"
        save_model(
            folder=output_folder,
            filename=filename,
            model=model_data["model"],
            train_loss=model_data["train_loss"],
            train_accs=model_data["train_accs"],
            test_loss=model_data["test_loss"],
            test_accs=model_data["test_accs"],
            additional_data=model_data["additional_data"],
            dataset_type=model_data["dataset_type"],
        )
        print(f"Saved model: {output_folder}/{filename}")

✅ Dataset saved to models_and_data\dataset.pt
Saved dataset to models_and_data/dataset.pt
✅ Model saved to models_and_data\model_additional_0.pt
Saved model: models_and_data/model_additional_0.pt
✅ Model saved to models_and_data\model_additional_282.pt
Saved model: models_and_data/model_additional_282.pt
✅ Model saved to models_and_data\model_additional_752.pt
Saved model: models_and_data/model_additional_752.pt
✅ Model saved to models_and_data\model_additional_1693.pt
Saved model: models_and_data/model_additional_1693.pt
✅ Model saved to models_and_data\model_additional_2634.pt
Saved model: models_and_data/model_additional_2634.pt
✅ Model saved to models_and_data\model_additional_3104.pt
Saved model: models_and_data/model_additional_3104.pt
✅ Model saved to models_and_data\model_additional_3575.pt
Saved model: models_and_data/model_additional_3575.pt
✅ Model saved to models_and_data\model_additional_4516.pt
Saved model: models_and_data/model_additional_4516.pt
✅ Model saved to models_